<a href="https://colab.research.google.com/github/kmk4444/Langchain/blob/main/Chain.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Langchain - Chains**
- In Langchain, there are a lot of chains but we will examine two chains.
- First chain relates on RAG.
- Second chain relates on Function Calling which is answering our function.

**Requirements.txt**

In [2]:
!touch requirements.txt
!echo langchain >> requirements.txt
!echo langchain-openai >> requirements.txt
!echo langchain-google-genai >> requirements.txt
!echo langchain_experimental >> requirements.txt
!echo openai >> requirements.txt
!echo anthropic >> requirements.txt
!echo cohere >> requirements.txt
!echo streamlit >> requirements.txt
!echo python-dotenv >> requirements.txt
!echo beautifulsoup4 >> requirements.txt
!echo faiss-cpu >> requirements.txt
!echo pypdf >> requirements.txt
!echo unstructured >> requirements.txt
!echo networkx >> requirements.txt
!echo openpyxl >> requirements.txt
!echo rapidocr-onnxruntime >> requirements.txt

**terminal / bash komutu**

In [ ]:
pip install -r requirements.txt

In [4]:
#%%writefile chain.py
# First chain for RAG
#Create Stuff Documents Chain
from langchain_openai import ChatOpenAI
from langchain_core.documents import Document # In RAG, we use that for input and output data of document. Document.page_content and Document.metadata
from langchain_core.prompts import ChatPromptTemplate # It eliminate writing too long string
from langchain.chains.combine_documents import create_stuff_documents_chain # It is the most significant method.

import os
from dotenv import load_dotenv

#load_dotenv()
#my_key_openai = os.getenv("openai_apikey")
my_key_openai="---"

llm = ChatOpenAI(model="gpt-4-0125-preview",api_key=my_key_openai)

#ChatPromptTemplate creates string
prompt= ChatPromptTemplate.from_messages(
    [("system", "Burada ismi geçen kişilerin en sevdiği rengi tek tek yaz:\n\n{context}")]
)

#above prompt code means that
#prompt : f'''"Burada ismi geçen kişilerin en sevdiği rengi tek tek yaz:
#
#
#{context}
#'''
# also, we add System key.

docs = [
    Document(page_content="Gamze kırmızıyı sever ama sarıyı sevmez"),
    Document(page_content="Murat yeşili sever ama maviyi sevdiği kadar değil"),
    Document(page_content="Burak'a sorsan favori rengim yok der ama belli ki turuncu rengi seviyor")
] # our documents


chain_1 = create_stuff_documents_chain(llm,prompt)
print(chain_1.invoke({"context":docs})) # context is our docs.
#chain_1 provides to create a chain among llm, prompt and docs



- Gamze'nin en sevdiği renk: Kırmızı
- Murat'ın en sevdiği renk: Mavi
- Burak'ın en sevdiği renk: Turuncu


In [6]:
#Second chain
#Create OpenAI Function Runnable Chain
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate # it provides to arrange prompt string
from langchain_core.pydantic_v1 import BaseModel, Field # data formatting
from typing import Optional # data formatting
from langchain.chains.openai_functions import create_openai_fn_runnable #second chain

#data types for insan
class Insan(BaseModel): #İnsan sınıfı kalıtım yoluyla BaseModel sınıfından türetiliyor
  """Bir insan hakkında tanımlayıcı bilgiler"""
  isim: str = Field(...,description="Kişinin İsmi") # "..." means obligation
  yas: int = Field(...,description="Kisinin yaşı")
  meslek: Optional[str] = Field(None,description="Kişinin Mesleği")

#data types for sehir
class Sehir(BaseModel):
  """Bir şehir hakkında tanımlayıcı bilgiler"""

  isim: str = Field(...,description="Şehrin İsmi")
  plaka_no: str = Field(...,description="Şehrin plaka numarası")
  iklim: Optional[str] = Field(None,description="Şehrin iklimi")

my_key_openai="---"

llm = ChatOpenAI(model="gpt-4-0125-preview",api_key=my_key_openai)

# we created prompt template.There are three messages which one is from system and the others are from human(user).
prompt = ChatPromptTemplate.from_messages(
    [
        ("system","Sen varlıkları kaydetmek konusunda dünyanın en başarılı algoritmasısın"),
        ("human","Şu verdiğim girdideki varlıkları kaydetmek için gerekli fonksiyonlara çağrı yap: {input}"),
        ("human","İpucu: Doğru formatta yanıtladığından emin ol")
    ]
)
chain_2 = create_openai_fn_runnable([Insan,Sehir],llm,prompt)
print(chain_2.invoke({"input":"Aydın 34 yaşında, başarılı bir bilgisayar mühendisiydi."}))
print(chain_2.invoke({"input":"Aydın'da hava her zaman sıcaktır ve bu yüzden 09 plakalı araçlarda klima hep çalışmaktadır."}))

isim='Aydın' yas=34 meslek='bilgisayar mühendisi'
isim='Aydın' plaka_no='09' iklim='sıcak'
